## Building a K-Nearest Neighbours classifier from scratch

<br>

### Introduction

The K-Nearest Neighbours (KNN) model provides a simple example of machine learning, in which unknown values can be classified using the known classifications of similar values.

This notebook provides a step-by-step process of readying a dataset for input, building a KNN classifier and assessing it's performance without the use of sklearn's pre-built equivalents.

__Theory__

KNN relies on the fact that items in the same category tend to have similar characteristics, so by determining which items of known category have the most similar characteristics to an unknown item, its category can be predicted.

Distance measures are used to determine the 'closeness' of items, in which the differences between each of their characteristics are used to calculate a single measure of distance. While many different distance measures exist for different situations, this example will use _Euclidean distance_, the formula for which is as follows:

$D(a,b) = \sqrt{\sum_{i=1}^{n}(b_{i}-a_{i})^{2}}$

In plain english; the distance between item a and item b is equal to the square root of the sum of the squared differences between each item's characteristics

In [470]:
import seaborn as sns
import pandas as pd
import numpy as np
import random

In [471]:
random.seed(1)

### Input dataset

The input data for a KNN classifier includes two parts:
- A categorical target/label variable which we would like to predict using the model
- Multiple feature variables, the values of which vary between categories of the target variable.

This seaborn sample dataset which we will use for our model includes 4 feature variables describing the dimensions of iris petals and sepals, as well a target variable specifying the species.

Before the dataset can be used, the all variables must be converted to a numeric format to allow calculations to be performed across the whole dataset. Therefore, our three iris species must be converted to 0, 1 and 2.

In [472]:
iris = sns.load_dataset('iris')
print(iris.shape)
iris.head()

(150, 5)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [1]:
#ordinal encoding
iris.species = pd.Categorical(iris.species)
iris['species_numeric'] = iris.species.cat.codes

NameError: name 'pd' is not defined

In [474]:
print(list(iris.species.unique()))
print(iris.species_numeric.unique())

['setosa', 'versicolor', 'virginica']
[0 1 2]


In [475]:
iris_data = iris.iloc[:, 0:4].to_numpy()
iris_labels = iris.species_numeric.to_numpy()

<br>

---

<br>

### Normalisation

Distance measures treat each feature variable equally when calculating the distance between two values. Therefore a mismatch in scale would result in the differences in one feature variable having a much larger impact on distance than another.

Min-max normalising the data solves this problem by converting all feature variables to the same scale from 0-1.

In [476]:
def normalise(data):
    #iterating through each column, min-max normalising all values
    for column in range(data.shape[1]):
        data[:,column] = (data[:,column] - data[:,column].min()) / (data[:,column].max() - data[:,column].min())
    return data

### Train-test split

In order to assess the performance of the model, we need to partition our data into training and testing groups. 

If we tested and tweaked the model using the same data used to build it, there is a much greater chance of overfitting, in which the model performance very well on the current data, but poorly on new data it has never seen before.

In [477]:
def train_test_split(data, labels):
    
    #creating a randomly ordered range the length of the data
    indices = list(range(len(data)))
    random.shuffle(indices)
    split = int(len(data) * 0.75)
    
    #selecting the first 75% of the randomly ordered data
    train_data = data[indices[:split]]
    train_labels = labels[indices[:split]]
    
    #selecting the remaining 25%
    test_data = data[indices[split:]]
    test_labels = labels[indices[split:]]
    
    return train_data, test_data, train_labels, test_labels

### K-nearest classifier

The KNN classifier class has been built as a simplified version sklearn's KNN classifier.

Step-by-step it functions as follows:
1. The 'fit' method is used to create instance variables for the characteristics (train_data) and category (train_labels) of items in our training group. 


2. The 'predict' method takes in the characteristics of items in our testing group (test_data) and predicts their category by finding the most common category of the 'K' closest items in our training group. The 'K' number adjusts how many neighbouring items are used to make a classification.


3. The 'score' method calls upon the 'predict' method to predict the categories of every item in the testing group, and then compares the predictions to the actual categories of the testing group to calculate the accuracy of the model.

In [478]:
class K_nearest_classifier():
    class
    def __init__(self):
        self.train_data = None
        self.train_labels = None
    
    def fit(self, train_data, train_labels):
        self.train_data = train_data
        self.train_labels = train_labels
    
    def predict(self, test_data, k):
        
        #finds the euclidean distance between any two datapoints using each of their features
        def euclidean_distance(datapoint_1, datapoint_2):
            total = 0
            for x in range(len(datapoint_1)):
                total += (datapoint_1[x] - datapoint_2[x])**2
            return total ** 0.5
        
        predictions = []
        for test_datapoint in range(len(test_data)):
            
            #finds the distance between an unknown datapoint and all datapoints in the labelled training data
            neighbours = []
            for train_datapoint in range(len(self.train_data)):
                distance = euclidean_distance(test_data[test_datapoint], self.train_data[train_datapoint])
                label = train_labels[train_datapoint]
                neighbours.append([distance, label])

            #finds the 'k' closest training datapoints to the unknown datapoint
            neighbours.sort()
            neighbours = neighbours[0:k]

            #creating an array of zeros, the indices of which represent each category in the dataset
            num_categories = len(set(train_labels))
            count = np.zeros(num_categories)

            #iterating through each neighbour's category, adding 1 to the corresponding indice, returning the most common
            for neighbour in neighbours:
                count[neighbour[1]] += 1

            predictions.append(count.argmax())
        
        return predictions
    
    def score(self, test_data, test_labels, k):
        
        predictions = self.predict(test_data, k)
        
        num_correct = 0.
        for x in range(len(predictions)):
            if predictions[x] == test_labels[x]:
                num_correct += 1
        return round(num_correct / len(test_labels), 3)

<br>

---

<br>

### Model testing  / assessing performance

In [479]:
iris_data = normalise(iris_data)
train_data, test_data, train_labels, test_labels = train_test_split(iris_data, iris_labels)

In [480]:
classifier = K_nearest_classifier()
classifier.fit(train_data, train_labels)

In [481]:
classifier.score(test_data, test_labels, 5)

0.947

In [483]:
print('Predicted categories:')
print(list(classifier.predict(test_data, 5))) 
print('True categories:')
print(list(test_labels))

Predicted categories:
[2, 0, 1, 2, 1, 1, 0, 1, 0, 2, 2, 0, 1, 2, 0, 0, 2, 1, 0, 1, 2, 2, 0, 2, 1, 0, 2, 0, 1, 1, 2, 2, 2, 0, 1, 0, 2, 0]
True categories:
[2, 0, 1, 2, 1, 1, 0, 1, 0, 2, 2, 0, 1, 1, 0, 0, 2, 1, 0, 1, 1, 2, 0, 2, 1, 0, 2, 0, 1, 1, 2, 2, 2, 0, 1, 0, 2, 0]
